# Quantum Fourier Transform (QFT) - Foundation of Quantum Algorithms

The Quantum Fourier Transform is a fundamental quantum algorithm that serves as the basis for many other important algorithms including Shor's factoring algorithm and quantum phase estimation. This notebook explores QFT implementation and its behavior across different quantum simulators.

In [ ]:
# Import required libraries
import time

from ariadne import explain_routing, simulate
from ariadne.algorithms import AlgorithmParameters, QuantumFourierTransform

## Understanding the Quantum Fourier Transform

The QFT is the quantum analogue of the discrete Fourier transform:

$QFT|x\rangle = \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1} e^{2\pi i xy/N} |y\rangle$

where $N = 2^n$ for n qubits.

Key properties:
- Transforms computational basis states to Fourier basis
- Provides exponential speedup over classical FFT
- Essential for phase estimation and period finding

## Creating QFT Circuits with Different Sizes

In [ ]:
# Create QFT circuits of different sizes
qft_sizes = [2, 3, 4, 5]
qft_circuits = {}

for n_qubits in qft_sizes:
    params = AlgorithmParameters(n_qubits=n_qubits)
    qft = QuantumFourierTransform(params)
    circuit = qft.create_circuit()
    qft_circuits[n_qubits] = circuit

    print(f"QFT-{n_qubits}: {circuit.num_qubits} qubits, {circuit.depth()} depth")
    print(f"Gate counts: {circuit.count_ops()}")
    print()

## Visualizing QFT Circuit Structure

In [ ]:
# Display the 4-qubit QFT circuit
print("4-qubit QFT Circuit:")
print(qft_circuits[4].draw(output="text"))

## Testing QFT Across Different Backends

In [ ]:
# Test 4-qubit QFT across different backends
test_circuit = qft_circuits[4]
backends = ["stim", "qiskit", "mps", "tensor_network"]
results = {}

print("Testing QFT-4 across backends:")
print("=" * 50)

for backend in backends:
    try:
        start_time = time.time()
        result = simulate(test_circuit, shots=1000, backend=backend)
        end_time = time.time()

        results[backend] = {
            "time": end_time - start_time,
            "counts": result.counts,
            "backend_used": result.backend_used.value,
        }

        print(f"{backend:15} | {end_time - start_time:8.4f}s | {result.backend_used.value}")

    except Exception as e:
        print(f"{backend:15} | FAILED - {str(e)[:30]}")
        results[backend] = {"error": str(e)}

## Analyzing QFT Results

In [ ]:
# Analyze the distribution of measurement outcomes
print("\nQFT Measurement Outcomes:")
print("=" * 40)

for backend, data in results.items():
    if "error" not in data:
        print(f"\n{backend.upper()} Results:")
        counts = data["counts"]
        total_shots = sum(counts.values())

        # Sort by measurement outcome
        sorted_counts = sorted(counts.items(), key=lambda x: x[0])

        for outcome, count in sorted_counts[:8]:  # Show first 8 outcomes
            probability = count / total_shots
            print(f"  |{outcome}⟩: {count:4d} ({probability:.3f})")

        if len(sorted_counts) > 8:
            print(f"  ... and {len(sorted_counts) - 8} more outcomes")

## Understanding QFT Scaling Behavior

In [ ]:
# Test scaling behavior of QFT
scaling_results = {}
test_sizes = [2, 3, 4, 5]
test_backend = "qiskit"  # Use reliable backend for scaling test

print(f"QFT Scaling Analysis (backend: {test_backend}):")
print("=" * 50)
print(f"{'Qubits':<6} | {'Depth':<6} | {'Time (s)':<9} | {'Gate Count':<10}")
print("-" * 40)

for n_qubits in test_sizes:
    circuit = qft_circuits[n_qubits]

    try:
        start_time = time.time()
        result = simulate(circuit, shots=1000, backend=test_backend)
        end_time = time.time()

        gate_count = len(circuit.data)

        print(f"{n_qubits:<6} | {circuit.depth():<6} | {end_time - start_time:<9.4f} | {gate_count:<10}")

        scaling_results[n_qubits] = {"time": end_time - start_time, "depth": circuit.depth(), "gates": gate_count}

    except Exception as e:
        print(f"{n_qubits:<6} | FAILED - {str(e)[:20]}")

## QFT Circuit Analysis

In [ ]:
# Analyze circuit properties for different QFT sizes
print("\nQFT Circuit Properties:")
print("=" * 30)

for n_qubits in test_sizes:
    params = AlgorithmParameters(n_qubits=n_qubits)
    qft = QuantumFourierTransform(params)
    analysis = qft.analyze_circuit_properties()

    print(f"\nQFT-{n_qubits}:")
    print(f"  Qubits: {analysis['n_qubits']}")
    print(f"  Depth: {analysis['depth']}")
    print(f"  Total gates: {analysis['size']}")
    print(f"  Two-qubit gates: {analysis['two_qubit_gates']}")
    print(f"  Entanglement heuristic: {analysis['entanglement_heuristic']:.2f}")
    print(f"  Gate types: {list(analysis['gate_counts'].keys())}")

## Educational Content: QFT Mathematical Background

In [ ]:
# Get educational content about QFT
params = AlgorithmParameters(n_qubits=4)
qft = QuantumFourierTransform(params)
educational_content = qft.get_educational_content()

print("=== QFT Mathematical Background ===")
print(educational_content["mathematical_background"])

print("\n=== Implementation Notes ===")
print(educational_content["implementation_notes"])

print("\n=== Applications ===")
print(educational_content["applications"])

## Routing Analysis for QFT

In [ ]:
# Analyze how Ariadne routes QFT circuits
print("Routing Analysis for QFT-4:")
print("=" * 30)

explanation = explain_routing(test_circuit)
print(explanation)

## Key Takeaways

1. **Circuit Complexity**: QFT depth scales as O(n²) with n qubits
2. **Entanglement**: High entanglement due to many controlled-phase gates
3. **Backend Performance**: Different backends handle QFT differently based on their strengths
4. **Applications**: Foundation for phase estimation, Shor's algorithm, and quantum signal processing
5. **Measurement Distribution**: Uniform distribution expected for input |0⟩ state

The QFT demonstrates how quantum algorithms can achieve exponential speedups over classical counterparts, making it a cornerstone of quantum computing education and research.